# Pythonで動かして学ぶ機械学習入門
# 第二回 評判分析

## 0.前準備

python versionの確認します。<br>
current directoryとdata directoryの内容を確認します。<br>
jupyter notebookではシェルコマンドの文頭に"!"をつけるとそのシェルコマンドをnotebook上で実行することができます。<br> 

In [1]:
!python -V

Python 3.4.3


In [2]:
!ls

ML_2_2_normal.ipynb data


In [3]:
!ls ./data/

README tokens


## 1.dataの読み込みとモジュールのインポート

pyenvなどを用いているとpandasなどがimportできない場合がある。<br>
その可能性の１つとしてlocale（国毎に異なる単位）の設定不足があり得るので、ここではそれを明示的に操作する。<br>

In [4]:
import os

def set_locale():
    default = os.environ.get('LC_ALL')
    print( "Your default locale is", default )
    if default is None:
        os.environ.setdefault('LC_ALL', 'ja_JP.UTF-8')
        print( "Your locale is set as ja_JP.UTF-8" )

set_locale()

Your default locale is None
Your locale is set as ja_JP.UTF-8


今回使うデータファイルのパスをpythonのリストとして取得します。<br>
globはパス名を見つけたりparseしたりするモジュールです( http://docs.python.jp/3/library/glob.html )。<br>

In [5]:
import glob

DATA_PATH = "./data/tokens/"

neg_files = glob.glob( "{0}neg/*".format(DATA_PATH) )
pos_files = glob.glob( "{0}pos/*".format(DATA_PATH) )

取得したファイルパスの確認。

In [6]:
print(neg_files[0:2])
print(pos_files[0:2])

['./data/tokens/neg/cv000_tok-9611.txt', './data/tokens/neg/cv001_tok-19324.txt']
['./data/tokens/pos/cv000_tok-11609.txt', './data/tokens/pos/cv001_tok-10180.txt']


読み込みテスト

実際に文章を１つ読み込んでみて正しく読み込めているかを確認します。<br>
テキストの読み込みはエンコーディングの問題などでエラーが生じやすいので、慣れるまでは根気強くdebugしましょう。<br>

In [7]:
with open(neg_files[0], 'r', encoding='utf-8') as f:
    for line in f:
        print(line)

tristar / 1 : 30 / 1997 / r ( language , violence , dennis rodman ) cast : jean-claude van damme ; mickey rourke ; dennis rodman ; natacha lindinger ; paul freeman director : tsui hark screenplay : dan jakoby ; paul mones ripe with explosions , mass death and really weird hairdos , tsui hark's " double team " must be the result of a tipsy hollywood power lunch that decided jean-claude van damme needs another notch on his bad movie-bedpost and nba superstar dennis rodman should have an acting career . actually , in " double team , " neither's performance is all that bad . i've always been the one critic to defend van damme -- he possesses a high charisma level that some genre stars ( namely steven seagal ) never aim for ; it's just that he's never made a movie so exuberantly witty since 1994's " timecop . " and rodman . . . well , he's pretty much rodman . he's extremely colorful , and therefore he pretty much fits his role to a t , even if the role is that of an ex-cia weapons expert .

今回使うモジュールの情報をまとめておきます。<br>
- matplotlib : グラフなどの描写する
- pandas : dataframeでデータを扱い、集計や統計量算出などがすぐ実行できる
- collections : pythonで扱えるデータの型を提供する
- sys : ファイルサイズ取得などのシステム上の操作を行う
- numpy : 行列などの数学的オブジェクトを扱う
- sklearn.feature_extraction.DictVectorizer : 辞書データをベクトルの形に変換する
- sklearnのモデル : SVM, NB, RF
- sklearn.grid_search : パラメタの最適な組み合わせ見つける

In [8]:
import matplotlib
import pandas as pd

import collections
import sys
import numpy as np

from sklearn.feature_extraction import DictVectorizer

from sklearn import svm, naive_bayes
from sklearn.ensemble import RandomForestClassifier

from sklearn import grid_search

## 2.特徴ベクトルの作成

unigramを作成する関数を定義します。<br>
スペース区切りで単語を抽出し、その数をカウントする単純な関数となります。<br>

In [9]:
def get_unigram(file_paths):
    result = []
    for file in file_paths:
        with open(file, 'r', encoding='utf-8') as f:
            for line in f:
                words = line.strip().split()
                count_dicts = collections.Counter(words)
                result.append( dict(count_dicts) )
    return result

この関数を用いて、negative と positive 両方で unigram を作成します。<br>
得られた2つのリストを合わせてモデルのインプットとします。
リストの結合は "+" で実施できます。<br>
negative と positive は各700文ずつありますが、そのうちいくつを使うかをここで指定します。初期設定では全てのデータを使うことになっていますが、後の過程で memory 不足になるようでしたらこの数を減らしてください。<br>

ipython notebookでは %% をつけることでmagic commands ( https://ipython.org/ipython-doc/3/interactive/magics.html ) という便利なコマンドを実行できます。ここでは処理にかかる時間をセルに表示するコマンドを使用しています。

In [10]:
%%time

DATA_NUM = 700

unigrams_data = get_unigram(neg_files[:DATA_NUM]) + get_unigram(pos_files[:DATA_NUM])

CPU times: user 459 ms, sys: 131 ms, total: 590 ms
Wall time: 1.17 s


得られたunigram_dataを確認してみます。単語の出現数がカウントされていることが確認できます。<br>
合わせてそのデータサイズも確認してみます。

In [11]:
print( unigrams_data[0] )
print( "data size :", sys.getsizeof(unigrams_data) / 1000000, "[MB]" )

{'superstar': 1, 'world': 1, ';': 6, 'much': 4, 'circles': 1, 'for': 4, 'each': 1, 'frame': 1, 'over': 1, 'antwerp': 1, '>http': 1, 'headache-inducing': 1, 'teams': 1, 'weird-looking': 1, 'whole': 1, 'acting': 1, 'nba': 1, 'climax': 1, 'tipsy': 1, 'known': 1, 'leading': 1, 'save': 1, '"': 22, 'actually': 1, 'aim': 1, 'valuable': 1, ')': 6, 'out': 3, 'lite': 1, "rodman's": 1, 'loud': 1, 'camera': 1, 'work': 2, 'made': 1, 'hollywood': 1, 'tries': 2, 'from': 1, 'taken': 1, 'running': 1, 'though': 1, 'r': 1, 'the': 20, 'kicks': 1, 'by': 1, 'entertaining': 1, 'think': 1, "i've": 1, 'natacha': 2, 'mildly': 1, 'cast': 1, "1994's": 1, "quinn's": 1, 'rome': 1, 'care': 1, 'possesses': 1, 'them': 2, 'just': 3, '(': 6, 'result': 2, 'deadly': 1, 'needs': 2, 'tsui': 2, "there's": 1, 'quinn': 5, 'rub': 1, 'some': 2, 'two': 1, 'mines': 1, 'jpeck1@gl': 1, 'indulge': 1, 'jean-claude': 2, 'action': 1, 'wacky': 1, 'critic': 1, 'exuberantly': 1, 'if': 1, 'edu</a>': 1, 'timecop': 1, 'movie-bedpost': 1, 'tra

得られたデータを扱いやすい行列の形にします。<br>
各行が１つのレビューテキストで、各列が単語、要素がその単語の出現数というデータを作成します。<br>
ここでは scikit-learn で準備されている DictVectorizer という関数を使うことでそれが簡単に実行できます。<br>

In [12]:
%%time
vec = DictVectorizer()
feature_vectors_csr = vec.fit_transform( unigrams_data )

CPU times: user 983 ms, sys: 36.4 ms, total: 1.02 s
Wall time: 1.06 s


作成したデータを確認してみます。

In [13]:
feature_vectors_csr

<1400x44219 sparse matrix of type '<class 'numpy.float64'>'
	with 496525 stored elements in Compressed Sparse Row format>

行列の全成分は 60,000,000 要素くらいありますが、このうちのほとんどは 0 で 0 以外の値が入っているのは500,000程度です。<br>
この CSR matrix　というのはこのような疎行列をの 0 でない成分だけを保持する賢いものになっています。<br>

一方で 0 の成分を陽に保って普通の行列としてデータを保持することも可能です。<br>

In [14]:
feature_vectors = vec.fit_transform( unigrams_data ).toarray()
print( "input data dimension :", feature_vectors.shape )
print( feature_vectors[0] )
print( "data size :", sys.getsizeof(feature_vectors) / 1000000, "[MB]" )

input data dimension : (1400, 44219)
[  0.   0.  22. ...,   0.   0.   0.]
data size : 495.252912 [MB]


こちらはデータが非常に大きくなっていますが、これは 0 という成分を陽に保持しているためです。<br>
この段階で memory error が生じる場合は一度 kernel を　restart して DATA_NUM の数を減らしてください。<br>

## 3.ラベルデータの作成

今回扱うデータセットは全てに negative → 0, positive → 1 というラベルが振られています。<br>
特徴ベクトルはnegative sample 700文とpositive sample 700文を合わせて作ったものなので、0が700個と1が700個並んでいるベクトルを作成すればよいことになります。<br>

In [15]:
labels = np.r_[np.tile(0, DATA_NUM), np.tile(1, DATA_NUM)]

確認

In [16]:
print( labels[0], labels[DATA_NUM-1], labels[DATA_NUM], labels[2*DATA_NUM-1]  )

0 0 1 1


## 4.学習用データとテスト用データの作成方法

論文の記述によれば、データを偏りがないように3分割に分け、three fold cross validationで評価しています。<br>
ここでは乱数を生成して、データを3等分することで同様の状況を再現することにします。<br>

In [17]:
np.random.seed(7789)

shuffle_order = np.random.choice( 2*DATA_NUM, 2*DATA_NUM, replace=False )

確認

In [18]:
print( "length :", len(shuffle_order) )
print( "first 10 elements :", shuffle_order[0:10] )

length : 1400
first 10 elements : [1235 1232  910  162  343 1160  221  545 1112 1322]


データの偏りが生じていないかを確認します。<br>
明らかに偏りが生じてしまった場合は乱数のseedを設定し直します。<br>

In [19]:
one_third_size = int( 2*DATA_NUM / 3. )
print( "one third of the lengh :", one_third_size )

print( "# of '1' in 1st set :", np.sum( labels[ shuffle_order[:one_third_size] ]  ) )
print( "# of '1' in 2nd set :", np.sum( labels[ shuffle_order[one_third_size:2*one_third_size] ]  ) )
print( "# of '1' in 3rd set :", np.sum( labels[ shuffle_order[2*one_third_size:] ]  ) )

one third of the lengh : 466
# of '1' in 1st set : 227
# of '1' in 2nd set : 233
# of '1' in 3rd set : 240


## 5.モデルを学習して精度を検証

学習に必要な関数を定義します。<br>
ここではモデルとして{Support Vector Machine, Naive Bayes, Random Forest}を用います。<br>
モデルの性能測定は予測と答えが一致する数をカウントして正答率を求めることで実施します。<br>

In [20]:
def N_splitter(seq, N):
    avg = len(seq) / float(N)
    out = []
    last = 0.0
    
    while last < len(seq):
        out.append(seq[int(last):int(last + avg)])
        last += avg
        
    return np.array(out)

In [21]:
def train_model(features, labels, method='SVM', parameters=None):
    ### set the model
    if method == 'SVM':
        model = svm.SVC()
    elif method == 'NB':
        model = naive_bayes.GaussianNB()
    elif method == 'RF':
        model = RandomForestClassifier()
    else:
        print("Set method as SVM (for Support vector machine), NB (for Naive Bayes) or RF (Random Forest)")
    ### set parameters if exists
    if parameters:
        model.set_params(**parameters)
    ### train the model
    model.fit( features, labels )
    ### return the trained model
    return model

def predict(model, features):
    predictions = model.predict( features )
    return predictions

def evaluate_model(predictions, labels):
    data_num = len(labels)
    correct_num = np.sum( predictions == labels )
    return data_num, correct_num

def cross_validate(n_folds, feature_vectors, labels, shuffle_order, method='SVM', parameters=None):
    result_test_num = []
    result_correct_num = []
    
    n_splits = N_splitter( range(2*DATA_NUM), n_folds )

    for i in range(n_folds):
        print( "Executing {0}th set...".format(i+1) )
        test_elems = shuffle_order[ n_splits[i] ]
        train_elems = np.array([])
        train_set = n_splits[ np.arange(n_folds) !=i ]
        for j in train_set:
            train_elems = np.r_[ train_elems, shuffle_order[j] ]
        train_elems = train_elems.astype(np.integer)

        # train
        model = train_model( feature_vectors[train_elems], labels[train_elems], method, parameters )
        # predict
        predictions = predict( model, feature_vectors[test_elems] )
        # evaluate
        test_num, correct_num = evaluate_model( predictions, labels[test_elems] )
        result_test_num.append( test_num )
        result_correct_num.append( correct_num )
    
    return result_test_num, result_correct_num

上記関数はCross validationの数を変数として設定できるように作ってあります。<br>
今回は 3-folds で分析を行います。<br>

In [22]:
N_FOLDS = 3

準備ができたのでここでモデルを学習してその精度を確認してみましょう！<br>
とりあえず何も考えずに準備した Bag Of Words をインプットにしてモデルを学習し、その精度を確認してみます。<br>

In [23]:
%%time
ans,corr = cross_validate(N_FOLDS, feature_vectors_csr, labels, shuffle_order, method='SVM', parameters=None)

Executing 1th set...
Executing 2th set...
Executing 3th set...
CPU times: user 14.3 s, sys: 238 ms, total: 14.6 s
Wall time: 15.8 s


結果を確認します。

In [24]:
print( "average precision", np.around( 100*sum(corr)/sum(ans), decimals=1 ), "%" )

average precision 63.2 %


**！！！具体的なテキスト確認を入れる！！！**

## 6.パラメタチューニング

grid searchでパラメタをチューニングすることでどの程度精度が上がるかを確認してみます。<br>

In [25]:
%%time

search_parameters = [
    {'kernel': ['rbf'], 'gamma': [1e-2, 1e-3, 1e-4], 'C': [0.1, 1, 10, 100, 1000]},
    {'kernel': ['linear'], 'C': [0.1, 1, 10, 100, 1000]}
]

model = svm.SVC()
clf = grid_search.GridSearchCV(model, search_parameters)
clf.fit( feature_vectors_csr, labels )

CPU times: user 4min 56s, sys: 4.03 s, total: 5min
Wall time: 5min 20s


In [26]:
%%time
params = clf.best_params_

ans,corr = cross_validate(N_FOLDS, feature_vectors_csr, labels, shuffle_order, method='SVM', parameters=params)

print( "average precision", np.around( 100*sum(corr)/sum(ans), decimals=1 ), "%" )

Executing 1th set...
Executing 2th set...
Executing 3th set...
average precision 79.5 %
CPU times: user 13.9 s, sys: 146 ms, total: 14 s
Wall time: 14.6 s


精度が15%程度も向上しました！機械学習においてモデルのパラメタチューニングが非常に重要であることが伺えます。

## 7.簡単な特徴量変換による効果の確認

論文に記載してあるように、Bag Of Words のカウント数を全て1にしてみることで精度にどのような変化が生じるかを確認します。

In [27]:
feature_vectors_csr.data[ feature_vectors_csr.data > 0 ] = 1.

変換したデータを用いて同様に学習プロセスを実行してみましょう。

In [28]:
%%time

ans, corr = cross_validate(N_FOLDS, feature_vectors_csr, labels, shuffle_order)

Executing 1th set...
Executing 2th set...
Executing 3th set...
CPU times: user 15 s, sys: 151 ms, total: 15.2 s
Wall time: 15.8 s


In [29]:
print( "average precision", np.around( 100*sum(corr)/sum(ans), decimals=1 ), "%" )

average precision 49.1 %


なんと精度がだいぶ落ちてしまいました。<br>
しかも現在の問題設定は 0 か 1 を判別するものなので、ランダムに判別するモデルでも 50% 程度になります。<br>
それと同程度ということは、そもそもモデルの学習が上手くいっていないのではないかということが疑われます。<br>
そのことを検証してみるために、もう一度パラメタチューニングを実施してみます。<br>

In [30]:
%%time

search_parameters = [
    {'kernel': ['rbf'], 'gamma': [1e-2, 1e-3, 1e-4], 'C': [0.1, 1, 10, 100, 1000]},
    {'kernel': ['linear'], 'C': [0.1, 1, 10, 100, 1000]}
]

model = svm.SVC()
clf = grid_search.GridSearchCV(model, search_parameters)
clf.fit( feature_vectors_csr, labels )

CPU times: user 4min 59s, sys: 3.1 s, total: 5min 2s
Wall time: 5min 16s


In [31]:
clf.best_params_

{'C': 10, 'gamma': 0.001, 'kernel': 'rbf'}

In [32]:
%%time

ans, corr = cross_validate(N_FOLDS, feature_vectors_csr, labels, shuffle_order, method='SVM', parameters=clf.best_params_)

Executing 1th set...
Executing 2th set...
Executing 3th set...
CPU times: user 15 s, sys: 204 ms, total: 15.2 s
Wall time: 16.4 s


In [33]:
print( "average precision", np.around( 100*sum(corr)/sum(ans), decimals=1 ), "%" )

average precision 82.7 %


パラメタを調整することで高い精度を発揮することが分かりました！<br>
この精度は論文に記載されているものとほぼ同程度のものとなっています。<br>

## 8.SVM以外のモデルを実行

Naive Bayes は sparse matrix 型のインプットを受け付けないので、numpy arrayとして作成したデータを入れなければなりません。

In [34]:
%%time

ans, corr = cross_validate(N_FOLDS, feature_vectors, labels, shuffle_order, method='NB')
print( "average precision", np.around( 100*sum(corr)/sum(ans), decimals=1 ), "%" )

Executing 1th set...
Executing 2th set...
Executing 3th set...
average precision 62.3 %
CPU times: user 4.26 s, sys: 4.25 s, total: 8.51 s
Wall time: 10.9 s


Random Forest も実行してみます。

In [35]:
%%time

ans, corr = cross_validate(N_FOLDS, feature_vectors, labels, shuffle_order, method='RF')
print( "average precision", np.around( 100*sum(corr)/sum(ans), decimals=1 ), "%" )

Executing 1th set...
Executing 2th set...
Executing 3th set...
average precision 65.8 %
CPU times: user 2.87 s, sys: 1.07 s, total: 3.94 s
Wall time: 4.3 s


これらの結果はパラメタチューニングをしていないものなので、興味がある方はパラメタによって結果がどう変わるかを調べてみてください。